In [ ]:
import os
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torchtext
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
import glob
import natsort
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

DATASET_ZIP_DIR = '/content/drive/MyDrive/EDM Using RNN.zip' # Setting the File location for training data. Here it should be set to the file location of BN-HTRD_BigData.zip

with zipfile.ZipFile(DATASET_ZIP_DIR,"r") as zip_ref:
    zip_ref.extractall("data")

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/EDM Using RNN/train.txt', sep= ';' ) #delimiter= ';'
i=0
#train_df.dataframeName = '/content/drive/MyDrive/EDM Using RNN/train.txt, sep '
nRow, nCol = train_df.shape
print(f'There are {nRow} rows and {nCol} columns')
train_df

There are 16000 rows and 2 columns


,text,labels
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [ ]:
val_df = pd.read_csv('/content/drive/MyDrive/EDM Using RNN/val.txt', sep= ';' ) #delimiter= ';'
nRow, nCol = val_df.shape
print(f'There are {nRow} rows and {nCol} columns')
val_df

There are 2000 rows and 2 columns


,text,labels
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/EDM Using RNN/test.txt', sep= ';' ) #delimiter= ';'
nRow, nCol = test_df.shape
print(f'There are {nRow} rows and {nCol} columns')
test_df

There are 2000 rows and 2 columns


,text,labels
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [ ]:
i=0
words = train_df.iloc[i,0]
word_list= words.split()
print(word_list)

['i', 'didnt', 'feel', 'humiliated']


In [ ]:
blank_token = 0
unknown_token =1
word2index={ '' '': 1}
wordcount={}
index2word={ 1: '' ''}
num= 2
for words in train_df['text']:
  word_list= words.split()
  for word in word_list:
    if word not in word2index:
      word2index[word]= num
      wordcount[word] = 1
      index2word[num]= word
      num+=1
    else:
       wordcount[word]+=1

for words in val_df['text']:
  word_list= words.split()
  for word in word_list:
    if word not in word2index:
      word2index[word]= num
      wordcount[word] = 1
      index2word[num]= word
      num+=1
    else:
       wordcount[word]+=1
print(word2index)
print(len(word2index))
print(index2word)


{'': 1, 'i': 2, 'didnt': 3, 'feel': 4, 'humiliated': 5, 'can': 6, 'go': 7, 'from': 8, 'feeling': 9, 'so': 10, 'hopeless': 11, 'to': 12, 'damned': 13, 'hopeful': 14, 'just': 15, 'being': 16, 'around': 17, 'someone': 18, 'who': 19, 'cares': 20, 'and': 21, 'is': 22, 'awake': 23, 'im': 24, 'grabbing': 25, 'a': 26, 'minute': 27, 'post': 28, 'greedy': 29, 'wrong': 30, 'am': 31, 'ever': 32, 'nostalgic': 33, 'about': 34, 'the': 35, 'fireplace': 36, 'will': 37, 'know': 38, 'that': 39, 'it': 40, 'still': 41, 'on': 42, 'property': 43, 'grouchy': 44, 'ive': 45, 'been': 46, 'little': 47, 'burdened': 48, 'lately': 49, 'wasnt': 50, 'sure': 51, 'why': 52, 'was': 53, 'taking': 54, 'or': 55, 'milligrams': 56, 'times': 57, 'recommended': 58, 'amount': 59, 'fallen': 60, 'asleep': 61, 'lot': 62, 'faster': 63, 'but': 64, 'also': 65, 'like': 66, 'funny': 67, 'as': 68, 'confused': 69, 'life': 70, 'teenager': 71, 'jaded': 72, 'year': 73, 'old': 74, 'man': 75, 'have': 76, 'with': 77, 'petronas': 78, 'for': 79, 

In [ ]:
vocab_size= len(word2index)

In [ ]:
# def word_to_tensor(text):
#   global word2index
#   word_list = text.split()
#   encoded = [word2index[word] for word in word_list]
#   t=torch.tensor(encoded)
#   pads =30-len(word_list)
#   tp =F.pad(t,(0,pads),"constant",1)
#   return tp


In [ ]:
def word_to_tensor(text):
  global word2index
  word_list = text.split()
  length=[]
  encoded= []
  for word in word_list:
    if word in word2index:
      index = word2index[word]
      #print(index)
    else:
       index=1
    encoded.append(index)
  length.append(len(word_list))
  length=torch.tensor(length, dtype= torch.long)
  t=torch.tensor(encoded, dtype= torch.long)
  t = t.type('torch.LongTensor')
  pads =100-len(word_list)
  tp =F.pad(t,(0,pads),"constant",0)
  return tp


In [ ]:
i = 37
text = train_df.iloc[i,0]
encoded_words = word_to_tensor(text)
print(encoded_words)

print(type(encoded_words))

tensor([  2,  95, 327, 328, 203,  12,  38,  39, 324,  94,  18,  22, 329, 203,
         12,   4, 330, 186, 198, 331, 332, 333, 334,   8, 335,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0])
<class 'torch.Tensor'>


In [ ]:
i = 37
text = train_df.iloc[i,0]
encoded_words = word_to_tensor(text)
print(encoded_words)

tensor([  2,  95, 327, 328, 203,  12,  38,  39, 324,  94,  18,  22, 329, 203,
         12,   4, 330, 186, 198, 331, 332, 333, 334,   8, 335,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0])


In [ ]:
emotions = {'anger': 0 , 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}

In [ ]:
emotion_list = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
N_labels = len(emotion_list)
def label_to_index(label):
  global emotion_list
  w=0
  for i in emotion_list:
    if i == label:
       return w
    else:
      w+=1

def label_to_tensor(letter):
    tensor = torch.zeros(1, N_labels)
    tensor[0][label_to_index(letter)] = 1
    return tensor

In [ ]:
i = 0
label = train_df.iloc[i,1]
encoded_labels = label_to_tensor(label)
print(encoded_labels)

tensor([[0., 0., 0., 0., 1., 0.]])


In [ ]:
class CustomDataset(Dataset):
  def __init__(self, filename):
     self.filename= filename
  def __len__(self):
    self.df = pd.read_csv( self.filename, sep= ';' )
    nRow, nCol = self.df.shape
    return nRow
  def __getitem__(self, idx):
        self.df = pd.read_csv( self.filename, sep= ';' )
        data = self.df.iloc[idx,0]
        data_tensor= word_to_tensor(data)
        #embeds = nn.Embedding(vocab_size, 50)
        #lookup_tensor = torch.tensor([word2index["data"]], dtype=torch.long)
        label= self.df.iloc[idx,1]
        label = label_to_index(label)
        #label= label_to_tensor(label)
        return (data_tensor,label)




In [ ]:
train_set = '/content/drive/MyDrive/EDM Using RNN/train.txt'
val_set= '/content/drive/MyDrive/EDM Using RNN/val.txt'
test_set= '/content/drive/MyDrive/EDM Using RNN/test.txt'

In [ ]:
train_dataset= CustomDataset(train_set)
val_dataset = CustomDataset(val_set)
test_dataset= CustomDataset(test_set)

In [ ]:
train_dataset[0]

(tensor([2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 4)

In [ ]:
val_dataset[5]

(tensor([   2,  674,   12,    4,  257, 3633,  102,  322,  185,   81,  676,   79,
          176,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 2)

In [ ]:
test_dataset[7]

(tensor([   2,  249,   52,    2, 8112,   12,   26, 1125,   77,   26, 2807,   19,
           53,  123,  410, 1253, 1511,   21,    1,  778,   35, 3002,    2,  718,
           76,   46,    9,   79,  780,  310,  228,   35, 9755, 3860,  213,   35,
         4991,  102, 1871,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 2)

In [ ]:
batch_size = 100
#batch_size_V = 73

train_loader = DataLoader(train_dataset, batch_size, shuffle=True ,num_workers=1, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size ,num_workers=1, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size ,num_workers=1, pin_memory=True)

In [ ]:
len(train_loader)


160

In [ ]:
len(val_loader)

20

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')


    else:
        return torch.device('cpu') #torch.device('cpu')


In [ ]:
device = get_default_device()
device

device(type='cuda')

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
def encoded_data(data):
      global word2index
      global index2word
      length = []
      result = []
      for text in data:
        encoded= []
        text_list = text.split()
        length.append(len(text_list))
        for word in text_list:
              if word in text_list:
                  index = word2index[word]
                  encoded.append(index)
              else:
                  index = 1
                  encoded.append(index)
              result.append(encoded)

      output_tensor = result
      print(length)
      #print(output_tensor)
      return (torch.IntTensor(output_tensor), torch.IntTensor(length))

In [ ]:
def length_data(data):
      global word2index
      global index2word
      length = []
      for text in data:
        #print(text)
        text_list = text.split()
        length.append(len(text_list))
      #print(output_tensor)
      return (torch.IntTensor(length))

In [ ]:
class TextClassificationBase(nn.Module):
    global word2index
    def training_step(self, batch):
        data, labels= batch
        data = to_device(data, device)
        labels= to_device(labels, device)
        #length= length_data(words)
        #print(data)
        #data, length= encoded_data(data)
        #print(data)
        #data = data[:,None,:]
        #data = data.float()
        out = self(data)                # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        data, labels= batch
        data = to_device(data, device)
        labels= to_device(labels, device)
        #length= length_data(words)

        #print(length)
        #print(data)
        #data, length= encoded_data(data)
        #print(data)
        #data = data[:,None,:]
        #data = data.float()
        out = self(data)                   # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class SentimentClassifier(TextClassificationBase):
    def __init__(self, vocab_size, embed_dim,
                 hidden, n_label, n_layers):
        super(SentimentClassifier, self).__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden,
                            num_layers=n_layers,
                            bidirectional=True,
                            batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden * 2, n_label)
    def forward(self, input):
        #print(input.size())
        embed_out = self.embed(input)
        #print(embed_out.size())
        hidden = torch.zeros(self.n_layers * 2 ,
                             input.shape[0], self.hidden)
        hidden = to_device(hidden, device)
        cell = torch.zeros( self.n_layers * 2,
                            input.shape[0], self.hidden)
        cell = to_device(cell, device)
        out_lstm, (hidden, cell) = self.lstm(embed_out,
                                             (hidden, cell))#dropout
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]),dim=1)
        out = self.fc(hidden)
        return out

In [ ]:
VOCAB_SIZE = vocab_size + 1
EMBEDDING_DIM = 150
HIDDEN= 64
NUM_LABEL = 6 # number of classes
NUM_LAYERS = 2
BATCH=100
RNN_Model = SentimentClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)
to_device(RNN_Model, device)

SentimentClassifier(
  (embed): Embedding(16198, 150)
  (lstm): LSTM(150, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
class GRUEmotionClassifier(TextClassificationBase):
    def __init__(self, vocab_size, embed_dim,
                 hidden, n_label, n_layers):
        super(GRUEmotionClassifier, self).__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden,
                            num_layers=n_layers,
                            bidirectional=True,
                            batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden * 2, n_label)
    def forward(self, input):
        #print(input.size())
        embed_out = self.embed(input)
        #print(embed_out.size())
        hidden = torch.zeros(self.n_layers * 2 ,
                             input.shape[0], self.hidden)
        hidden = to_device(hidden, device)
        out_gru, hidden = self.gru(embed_out,
                                             hidden)#dropout
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]),dim=1)
        out = self.fc(hidden)
        return out

In [ ]:
VOCAB_SIZE = vocab_size + 1
EMBEDDING_DIM = 150
HIDDEN= 64
NUM_LABEL = 6 # number of classes
NUM_LAYERS = 2
BATCH=200
GRU_Model = GRUEmotionClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)
to_device(GRU_Model, device)

GRUEmotionClassifier(
  (embed): Embedding(16198, 150)
  (gru): GRU(150, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader,FILE, model_name ,opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        torch.save(model.state_dict(), os.path.join(FILE, model_name ))
    return history

In [ ]:
evaluate(RNN_Model, val_loader)

{'val_loss': 1.7624231576919556, 'val_acc': 0.33000001311302185}

In [ ]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 0.001
FILE = '/content/drive/MyDrive/EDM Using RNN/Models'
model_name= 'LSTM_EDM_15'
history = fit(num_epochs, lr, RNN_Model, train_loader, val_loader, FILE, model_name, opt_func)

Epoch [0], train_loss: 1.5369, val_loss: 1.3787, val_acc: 0.4790
Epoch [1], train_loss: 0.9654, val_loss: 0.8336, val_acc: 0.7190
Epoch [2], train_loss: 0.5239, val_loss: 0.5274, val_acc: 0.8105
Epoch [3], train_loss: 0.3354, val_loss: 0.4864, val_acc: 0.8380
Epoch [4], train_loss: 0.2148, val_loss: 0.4766, val_acc: 0.8465
Epoch [5], train_loss: 0.1596, val_loss: 0.4162, val_acc: 0.8730
Epoch [6], train_loss: 0.1132, val_loss: 0.4203, val_acc: 0.8805
Epoch [7], train_loss: 0.0830, val_loss: 0.4345, val_acc: 0.8785
Epoch [8], train_loss: 0.0672, val_loss: 0.4393, val_acc: 0.8865
Epoch [9], train_loss: 0.0598, val_loss: 0.5297, val_acc: 0.8640
Epoch [10], train_loss: 0.0588, val_loss: 0.4381, val_acc: 0.8885
Epoch [11], train_loss: 0.0467, val_loss: 0.4389, val_acc: 0.8895
Epoch [12], train_loss: 0.0407, val_loss: 0.4811, val_acc: 0.8910
Epoch [13], train_loss: 0.0583, val_loss: 0.4448, val_acc: 0.8905
Epoch [14], train_loss: 0.0330, val_loss: 0.4790, val_acc: 0.8885


In [ ]:
evaluate(GRU_Model, val_loader)

{'val_loss': 1.8013042211532593, 'val_acc': 0.061500001698732376}

In [ ]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 0.001
FILE = '/content/drive/MyDrive/EDM Using RNN/Models'
model_name= 'GRU_EDM_15'
history = fit(num_epochs, lr, GRU_Model, train_loader, val_loader, FILE, model_name, opt_func)

Epoch [0], train_loss: 1.5049, val_loss: 1.3530, val_acc: 0.4980
Epoch [1], train_loss: 0.8769, val_loss: 0.6167, val_acc: 0.7695
Epoch [2], train_loss: 0.4098, val_loss: 0.4502, val_acc: 0.8490
Epoch [3], train_loss: 0.2221, val_loss: 0.3358, val_acc: 0.8800
Epoch [4], train_loss: 0.1384, val_loss: 0.3421, val_acc: 0.8870
Epoch [5], train_loss: 0.0941, val_loss: 0.3125, val_acc: 0.8980
Epoch [6], train_loss: 0.0708, val_loss: 0.3156, val_acc: 0.9025
Epoch [7], train_loss: 0.0579, val_loss: 0.3172, val_acc: 0.9080
Epoch [8], train_loss: 0.0469, val_loss: 0.3614, val_acc: 0.8980
Epoch [9], train_loss: 0.0394, val_loss: 0.3462, val_acc: 0.9055
Epoch [10], train_loss: 0.0327, val_loss: 0.3648, val_acc: 0.9085
Epoch [11], train_loss: 0.0270, val_loss: 0.3741, val_acc: 0.9080
Epoch [12], train_loss: 0.0298, val_loss: 0.3836, val_acc: 0.9015
Epoch [13], train_loss: 0.0246, val_loss: 0.3700, val_acc: 0.9075
Epoch [14], train_loss: 0.0199, val_loss: 0.3728, val_acc: 0.9110


In [ ]:
FILE = "/content/drive/MyDrive/EDM Using RNN/Models/RNN_EDM_60"
model = SentimentClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)
model.load_state_dict(torch.load(FILE))
to_device(model, device)

SentimentClassifier(
  (embed): Embedding(16198, 150)
  (lstm): LSTM(150, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
)

In [ ]:
evaluate(model, test_loader)

{'val_loss': 0.4727708399295807, 'val_acc': 0.8854999542236328}

In [ ]:
FILE = "/content/drive/MyDrive/EDM Using RNN/Models/GRU_EDM_15"
model = GRUEmotionClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)
model.load_state_dict(torch.load(FILE))
to_device(model, device)

GRUEmotionClassifier(
  (embed): Embedding(16198, 150)
  (gru): GRU(150, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
evaluate(model, test_loader)

{'val_loss': 0.4929075241088867, 'val_acc': 0.8855000734329224}

In [ ]:
FILE = "/content/drive/MyDrive/EDM Using RNN/Models/LSTM_EDM_15"
model = SentimentClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN, NUM_LABEL, NUM_LAYERS)
model.load_state_dict(torch.load(FILE))
to_device(model, device)

SentimentClassifier(
  (embed): Embedding(16198, 150)
  (lstm): LSTM(150, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
evaluate(model, test_loader)

{'val_loss': 0.45655956864356995, 'val_acc': 0.8839999437332153}